In [2]:
import numpy as np
import chainer
from chainer.backends import cuda
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

from chainer.datasets import split_dataset_random

## Load MNIST dataset

In [3]:
from chainer.datasets import mnist

train_valid, test = mnist.get_mnist()

In [4]:
print("train_valid: %s" % type(train_valid))
print("test: %s" % type(test))

print("# of train_valid: %s" % len(train_valid))
print("# of test: %s" % len(test))

train_valid: <class 'chainer.datasets.tuple_dataset.TupleDataset'>
test: <class 'chainer.datasets.tuple_dataset.TupleDataset'>
# of train_valid: 60000
# of test: 10000


### Split the dataset

In [6]:
train, valid = split_dataset_random(train_valid, 50000, seed = 0)

In [7]:
print("# of training dataset: ",  len(train))
print("# of validation dataset: ", len(valid))
print("# of test dataset: ", len(test))

# of training dataset:  50000
# of validation dataset:  10000
# of test dataset:  10000


## Prepare iterators

In [8]:
batch_size = 128

train_iter = iterators.SerialIterator(train, batch_size)
valid_iter = iterators.SerialIterator(valid, batch_size, repeat=False, shuffle=False)
test_iter  = iterators.SerialIterator(test, batch_size, repeat=False, shuffle=False)

## Build the model

In [26]:
class ConvNet(Chain):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        with self.init_scope():
            self.c1 = L.Convolution2D(1, 6, ksize=5, stride=1, pad=2)  # same padding            
    
    def __call__(self, x):
        X = F.reshape(x, (-1, 1, 28, 28))

        h1 = F.tanh(self.c1(X))
        return h1

In [27]:
x0, y0 = train[0]
model = ConvNet()
print(type(model(train[0:])))

print(model(x0).shape)

<class 'chainer.variable.Variable'>
(1, 6, 28, 28)


## Create trainer

In [18]:
def create_trainer(model,  train_iter, valid_iter, max_epoch=10, gpu_id = -1):
    if gpu_id >= 0:
        model.to_gpu(gpu_id)

    optimizer = optimizers.SGD()
    optimizer.setup(model)
    
    updater = training.updater.StandardUpdater(train_iter, optimizer, device=gpu_id)

    trainer = training.Trainer(updater, (max_epoch, 'epoch'), out='.out/mnist_result')
    
    trainer.extend(extensions.LogReport())
    trainer.extend(extensions.snapshot(filename='snapshot_epoch-{.updater.epoch}'))
    trainer.extend(extensions.snapshot_object(model.predictor, filename='model_epoch-{.updater.epoch}'))
    trainer.extend(extensions.Evaluator(valid_iter, model, device=gpu_id))
    trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'main/accuracy', 'validation/main/loss', 'validation/main/accuracy', 'elapsed_time']))
    trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], x_key='epoch', file_name='loss.png'))
    trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'], x_key='epoch', file_name='accuracy.png'))
    trainer.extend(extensions.dump_graph('main/loss'))
    
    return trainer

In [16]:
model = ConvNet()
model = L.Classifier(model)

train_iter.reset()
valid_iter.reset()
test_iter.reset()

trainer = create_trainer(model, train_iter, valid_iter, max_epoch=50)
trainer.run()

NameError: name 'MLP' is not defined

## Evaluate test set

In [15]:
gpu_id = -1

test_evaluator = extensions.Evaluator(test_iter, model, device=gpu_id)
results = test_evaluator()
print('Test accuracy:', results['main/accuracy'])

NameError: name 'model' is not defined